# GNOD Song Suggester

## Web Scraping Hot 100 from Billboard

 Scraping the current top 100 songs and their respective artists from https://www.billboard.com/charts/hot-100, and putting the information into a pandas dataframe.

In [1]:
# importing libraries

from bs4 import BeautifulSoup
import re 
import requests
import pandas as pd
from tqdm.notebook import tqdm
import random
from random import randint
import warnings
warnings.filterwarnings("ignore")

### Getting the info from the web

In [2]:
# storing link in a variable
url = "https://www.billboard.com/charts/hot-100"
# downloading  html with a get request
response = requests.get(url)
# check response status code 
response.status_code

200

In [3]:
# parsing and storing the contents of the url call
billboardsoup = BeautifulSoup(response.content, 'html.parser')

In [4]:
# looping the songs
top_100 = len(billboardsoup.select('h3.c-title.a-no-trucate'))
song = []
artist = []

for i in tqdm(range(top_100)):
    song.append(billboardsoup.select('h3.c-title.a-no-trucate')[i].get_text(strip=True))
    artist.append(billboardsoup.select('span.c-label.a-font-primary-s')[i].get_text(strip=True))

  0%|          | 0/100 [00:00<?, ?it/s]

### Getting the dataframe and storing it

In [5]:
billboard_top100 = pd.DataFrame({'song':song,'artist':artist})

## Spotipy (API Wrappers)

In [6]:
# more libraries, you can never have enough

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import getpass
import pprint

In [7]:
client_id = str(getpass.getpass('client_id: '))
client_secret = str(getpass.getpass('client_secret: '))
# embedding getpass credentials in the access key
sp = spotipy.Spotify(auth_manager = SpotifyClientCredentials(client_id = client_id, client_secret = client_secret))

client_id: ········
client_secret: ········


### Choosing some playlist to create a dataframe

### Supernatural soundtrack

In [8]:
first_playlist = sp.user_playlist_tracks("solitude collective", "1IEQ8C3G1qT0W80muYgROT")

In [9]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
first_results = get_tracks("solitude collective", "1IEQ8C3G1qT0W80muYgROT")

In [10]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = first_results
for r in first_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [11]:
features_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[199:299]))
features_info4 = pd.DataFrame(sp.audio_features(tracks = song_id[299:399]))
features_info5 = pd.DataFrame(sp.audio_features(tracks = song_id[399:]))
features = pd.concat([features_info, features_info2, features_info3, features_info4, features_info5])
features.reset_index(drop=True, inplace=True)

In [12]:
first_playlist = pd.concat([playlist_info, features], axis=1)
first_playlist.reset_index()

,index,song_name,song_id,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,Carry on Wayward Son,2SUpC3UgKwLVOS2FtZif9N,Kansas,0.452,0.788,7,-5.712,1,0.0326,...,0.4410,0.7930,126.739,audio_features,2SUpC3UgKwLVOS2FtZif9N,spotify:track:2SUpC3UgKwLVOS2FtZif9N,https://api.spotify.com/v1/tracks/2SUpC3UgKwLV...,https://api.spotify.com/v1/audio-analysis/2SUp...,322093,4
1,1,Jump,2PfBSkCKZDXMsJogMT0wML,Van Halen,0.547,0.822,0,-7.201,1,0.0292,...,0.0645,0.7740,130.138,audio_features,2PfBSkCKZDXMsJogMT0wML,spotify:track:2PfBSkCKZDXMsJogMT0wML,https://api.spotify.com/v1/tracks/2PfBSkCKZDXM...,https://api.spotify.com/v1/audio-analysis/2PfB...,239560,4
2,2,Rock You Like A Hurricane,0XIvZ82aDF7JiSi3ZE320u,Scorpions,0.482,0.601,7,-12.846,1,0.0486,...,0.2490,0.7740,125.619,audio_features,0XIvZ82aDF7JiSi3ZE320u,spotify:track:0XIvZ82aDF7JiSi3ZE320u,https://api.spotify.com/v1/tracks/0XIvZ82aDF7J...,https://api.spotify.com/v1/audio-analysis/0XIv...,252493,4
3,3,Back In Black,08mG3Y1vljYA6bvDt4Wqkj,AC/DC,0.310,0.700,9,-5.678,1,0.0470,...,0.0828,0.7630,188.386,audio_features,08mG3Y1vljYA6bvDt4Wqkj,spotify:track:08mG3Y1vljYA6bvDt4Wqkj,https://api.spotify.com/v1/tracks/08mG3Y1vljYA...,https://api.spotify.com/v1/audio-analysis/08mG...,255493,4
4,4,Hot Blooded,6xMHglHoafdDFGXS6qfwSH,Foreigner,0.705,0.806,0,-4.830,1,0.0814,...,0.0676,0.7630,117.945,audio_features,6xMHglHoafdDFGXS6qfwSH,spotify:track:6xMHglHoafdDFGXS6qfwSH,https://api.spotify.com/v1/tracks/6xMHglHoafdD...,https://api.spotify.com/v1/audio-analysis/6xMH...,268693,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,424,I'd Be Lost If It Wasn't for You,64FhrwziLZYXXTtmhgk9Xh,Elijah Honey,0.639,0.743,5,-7.198,1,0.0527,...,0.3950,0.8490,120.415,audio_features,64FhrwziLZYXXTtmhgk9Xh,spotify:track:64FhrwziLZYXXTtmhgk9Xh,https://api.spotify.com/v1/tracks/64FhrwziLZYX...,https://api.spotify.com/v1/audio-analysis/64Fh...,181893,4
425,425,My Curse,6zKF4293k44ItKWJJgrhXv,Killswitch Engage,0.197,0.956,10,-2.908,0,0.0663,...,0.2610,0.2090,177.018,audio_features,6zKF4293k44ItKWJJgrhXv,spotify:track:6zKF4293k44ItKWJJgrhXv,https://api.spotify.com/v1/tracks/6zKF4293k44I...,https://api.spotify.com/v1/audio-analysis/6zKF...,244773,3
426,426,Livin' On A Prayer,7qQnBfwXrw2tZNFG4Uf57N,Bon Jovi,0.520,0.887,0,-3.296,1,0.0349,...,0.1860,0.7930,122.528,audio_features,7qQnBfwXrw2tZNFG4Uf57N,spotify:track:7qQnBfwXrw2tZNFG4Uf57N,https://api.spotify.com/v1/tracks/7qQnBfwXrw2t...,https://api.spotify.com/v1/audio-analysis/7qQn...,250627,4
427,427,"Touch-A, Touch-A, Touch Me",2SK1kGbpABI9EyUbADSP5z,Susan Sarandon,0.355,0.351,1,-13.572,1,0.0621,...,0.1090,0.4310,171.896,audio_features,2SK1kGbpABI9EyUbADSP5z,spotify:track:2SK1kGbpABI9EyUbADSP5z,https://api.spotify.com/v1/tracks/2SK1kGbpABI9...,https://api.spotify.com/v1/audio-analysis/2SK1...,151922,4


### All out 80s

In [13]:
second_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DX4UtSsGT1Sbe")

In [14]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
second_results = get_tracks("Spotify", "37i9dQZF1DX4UtSsGT1Sbe")

In [15]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = second_results
for r in second_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist2_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [16]:
features2_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features2_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:]))
features2 = pd.concat([features2_info, features2_info2])
features2.reset_index(drop=True, inplace=True)

In [17]:
second_playlist = pd.concat([playlist2_info, features2], axis=1)
second_playlist.reset_index()

,index,song_name,song_id,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,Take on Me,2WfaOiMkCvy7F5fcp2zZ8L,a-ha,0.573,0.902,6,-7.638,0,0.0540,...,0.0928,0.876,84.412,audio_features,2WfaOiMkCvy7F5fcp2zZ8L,spotify:track:2WfaOiMkCvy7F5fcp2zZ8L,https://api.spotify.com/v1/tracks/2WfaOiMkCvy7...,https://api.spotify.com/v1/audio-analysis/2Wfa...,225280,4
1,1,A Groovy Kind Of Love,5huh0htdiJDatAtbHLtpfz,Phil Collins,0.460,0.182,2,-14.711,1,0.0303,...,0.1010,0.188,144.066,audio_features,5huh0htdiJDatAtbHLtpfz,spotify:track:5huh0htdiJDatAtbHLtpfz,https://api.spotify.com/v1/tracks/5huh0htdiJDa...,https://api.spotify.com/v1/audio-analysis/5huh...,209333,4
2,2,Time After Time,1Jj6MF0xDOMA3Ut2Z368Bx,Cyndi Lauper,0.724,0.436,0,-9.321,1,0.0282,...,0.0908,0.324,130.439,audio_features,1Jj6MF0xDOMA3Ut2Z368Bx,spotify:track:1Jj6MF0xDOMA3Ut2Z368Bx,https://api.spotify.com/v1/tracks/1Jj6MF0xDOMA...,https://api.spotify.com/v1/audio-analysis/1Jj6...,243067,4
3,3,What's Love Got to Do with It,4kOfxxnW1ukZdsNbCKY9br,Tina Turner,0.851,0.406,1,-11.631,1,0.0661,...,0.0853,0.792,97.840,audio_features,4kOfxxnW1ukZdsNbCKY9br,spotify:track:4kOfxxnW1ukZdsNbCKY9br,https://api.spotify.com/v1/tracks/4kOfxxnW1ukZ...,https://api.spotify.com/v1/audio-analysis/4kOf...,226880,4
4,4,Dancing In the Dark,7FwBtcecmlpc1sLySPXeGE,Bruce Springsteen,0.527,0.942,1,-5.640,0,0.0366,...,0.1880,0.495,148.723,audio_features,7FwBtcecmlpc1sLySPXeGE,spotify:track:7FwBtcecmlpc1sLySPXeGE,https://api.spotify.com/v1/tracks/7FwBtcecmlpc...,https://api.spotify.com/v1/audio-analysis/7FwB...,241307,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,145,I Just Called To Say I Love You,6RNDeRnWsRMjPdNVgupZCs,Stevie Wonder,0.748,0.551,1,-9.054,1,0.0239,...,0.0943,0.650,113.535,audio_features,6RNDeRnWsRMjPdNVgupZCs,spotify:track:6RNDeRnWsRMjPdNVgupZCs,https://api.spotify.com/v1/tracks/6RNDeRnWsRMj...,https://api.spotify.com/v1/audio-analysis/6RND...,262240,4
146,146,Sweet Little Mystery,58qBa4R0T7u0ZiuRvMJRUN,Wet Wet Wet,0.711,0.581,0,-14.108,1,0.0357,...,0.0709,0.949,117.084,audio_features,58qBa4R0T7u0ZiuRvMJRUN,spotify:track:58qBa4R0T7u0ZiuRvMJRUN,https://api.spotify.com/v1/tracks/58qBa4R0T7u0...,https://api.spotify.com/v1/audio-analysis/58qB...,224933,4
147,147,Mary's Prayer,6G3afsR0udhKnAU0Tut25j,Danny Wilson,0.682,0.571,0,-13.139,1,0.0367,...,0.1160,0.657,135.109,audio_features,6G3afsR0udhKnAU0Tut25j,spotify:track:6G3afsR0udhKnAU0Tut25j,https://api.spotify.com/v1/tracks/6G3afsR0udhK...,https://api.spotify.com/v1/audio-analysis/6G3a...,234467,4
148,148,Into the Groove,2QkjnJ6Q8w9zYln1io4S0C,Madonna,0.729,0.856,10,-4.164,0,0.0350,...,0.0691,0.804,116.467,audio_features,2QkjnJ6Q8w9zYln1io4S0C,spotify:track:2QkjnJ6Q8w9zYln1io4S0C,https://api.spotify.com/v1/tracks/2QkjnJ6Q8w9z...,https://api.spotify.com/v1/audio-analysis/2Qkj...,285093,4


### All time top 1000 songs

In [18]:
third_playlist = sp.user_playlist_tracks("Peter Nordestgaard", "0lNwm5xFEgRx4iDP2DLx3j")

In [19]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
third_results = get_tracks("Peter Nordestgaard", "0lNwm5xFEgRx4iDP2DLx3j")

In [20]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = third_results
for r in third_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist3_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [21]:
features3_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features3_info1 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features3_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[199:299]))
features3_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[299:399]))
features3_info4 = pd.DataFrame(sp.audio_features(tracks = song_id[399:499]))
features3_info5 = pd.DataFrame(sp.audio_features(tracks = song_id[499:599]))
features3_info6 = pd.DataFrame(sp.audio_features(tracks = song_id[599:699]))
features3_info7 = pd.DataFrame(sp.audio_features(tracks = song_id[699:799]))
features3_info8 = pd.DataFrame(sp.audio_features(tracks = song_id[799:899]))
features3_info9 = pd.DataFrame(sp.audio_features(tracks = song_id[899:999]))
features3_info10 = pd.DataFrame(sp.audio_features(tracks = song_id[999:]))
features3 = pd.concat([features3_info, features3_info1, features3_info2, features3_info3, features3_info4, features3_info5, features3_info6, features3_info7, features3_info8, features3_info9, features3_info10])
features3.reset_index(drop=True, inplace=True)

In [22]:
third_playlist = pd.concat([playlist3_info, features3_info], axis=1)
third_playlist.reset_index()

,index,song_name,song_id,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,Imagine - Remastered 2010,7pKfPomDEeI4TPT6EOYjn9,John Lennon,0.547,0.257,0.0,-12.358,1.0,0.0252,...,0.0935,0.169,75.752,audio_features,7pKfPomDEeI4TPT6EOYjn9,spotify:track:7pKfPomDEeI4TPT6EOYjn9,https://api.spotify.com/v1/tracks/7pKfPomDEeI4...,https://api.spotify.com/v1/audio-analysis/7pKf...,187867.0,4.0
1,1,A Whiter Shade Of Pale,3qitymULqEibr7yknRMKU4,Procol Harum,0.249,0.660,0.0,-6.905,1.0,0.0342,...,0.0891,0.435,149.813,audio_features,3qitymULqEibr7yknRMKU4,spotify:track:3qitymULqEibr7yknRMKU4,https://api.spotify.com/v1/tracks/3qitymULqEib...,https://api.spotify.com/v1/audio-analysis/3qit...,248947.0,4.0
2,2,My Sweet Lord,1chu8wc9zbNqTQ2ahROys1,George Harrison,0.537,0.685,1.0,-8.935,0.0,0.0333,...,0.0839,0.542,120.965,audio_features,1chu8wc9zbNqTQ2ahROys1,spotify:track:1chu8wc9zbNqTQ2ahROys1,https://api.spotify.com/v1/tracks/1chu8wc9zbNq...,https://api.spotify.com/v1/audio-analysis/1chu...,281227.0,4.0
3,3,God Only Knows - Mono / 1997 Remastered,6iGU74CwXuT4XVepjc9Emf,The Beach Boys,0.521,0.487,6.0,-7.525,0.0,0.0299,...,0.0579,0.483,117.072,audio_features,6iGU74CwXuT4XVepjc9Emf,spotify:track:6iGU74CwXuT4XVepjc9Emf,https://api.spotify.com/v1/tracks/6iGU74CwXuT4...,https://api.spotify.com/v1/audio-analysis/6iGU...,173040.0,4.0
4,4,Bridge Over Troubled Water,6l8EbYRtQMgKOyc1gcDHF9,Simon & Garfunkel,0.149,0.206,3.0,-13.888,1.0,0.0323,...,0.1150,0.264,79.764,audio_features,6l8EbYRtQMgKOyc1gcDHF9,spotify:track:6l8EbYRtQMgKOyc1gcDHF9,https://api.spotify.com/v1/tracks/6l8EbYRtQMgK...,https://api.spotify.com/v1/audio-analysis/6l8E...,293120.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,She Will Be Loved,7sapKrjDij2fpDVj0GxP66,Maroon 5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,996,Achy Breaky Heart,2EoIt9vdgFRNW03u5IvFsQ,Billy Ray Cyrus,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,997,Reet Petite,3h4udS0WeWbsur3yfjvnm4,Jackie Wilson,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,998,All I Want for Christmas Is You,0bYg9bo50gSsH3LtXe2SQn,Mariah Carey,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Modern Chill Rock

In [23]:
fourth_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DX2UXfvEIZvDK")

In [24]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
fourth_results = get_tracks("Spotify", "37i9dQZF1DX2UXfvEIZvDK")

In [25]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = fourth_results
for r in fourth_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist4_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [26]:
features4_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:]))

In [27]:
fourth_playlist = pd.concat([playlist4_info, features4_info], axis=1)
fourth_playlist.reset_index()

,index,song_name,song_id,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,Talk Tonight,1gANYFsOT41zu2cdSC06iD,Oasis,0.355,0.452,4,-8.420,0,0.0265,...,0.3710,0.3510,83.930,audio_features,1gANYFsOT41zu2cdSC06iD,spotify:track:1gANYFsOT41zu2cdSC06iD,https://api.spotify.com/v1/tracks/1gANYFsOT41z...,https://api.spotify.com/v1/audio-analysis/1gAN...,261107,4
1,1,Chasing Birds,2jKnUcyrCADkeSy6gcN1qa,Foo Fighters,0.620,0.532,2,-7.297,1,0.0230,...,0.1120,0.3690,143.993,audio_features,2jKnUcyrCADkeSy6gcN1qa,spotify:track:2jKnUcyrCADkeSy6gcN1qa,https://api.spotify.com/v1/tracks/2jKnUcyrCADk...,https://api.spotify.com/v1/audio-analysis/2jKn...,252293,4
2,2,Road Trippin',7Gpr3kKk4BMgItz6UbI73q,Red Hot Chili Peppers,0.378,0.474,4,-8.202,0,0.0271,...,0.1170,0.3800,102.379,audio_features,7Gpr3kKk4BMgItz6UbI73q,spotify:track:7Gpr3kKk4BMgItz6UbI73q,https://api.spotify.com/v1/tracks/7Gpr3kKk4BMg...,https://api.spotify.com/v1/audio-analysis/7Gpr...,204960,4
3,3,Fly Like an Eagle,0SiS1Iud2fNazdtccUxE7m,Stereophonics,0.515,0.407,0,-8.032,1,0.0278,...,0.0611,0.4160,146.020,audio_features,0SiS1Iud2fNazdtccUxE7m,spotify:track:0SiS1Iud2fNazdtccUxE7m,https://api.spotify.com/v1/tracks/0SiS1Iud2fNa...,https://api.spotify.com/v1/audio-analysis/0SiS...,272587,3
4,4,Karma Police,3SVAN3BRByDmHOhKyIDxfC,Radiohead,0.360,0.505,7,-9.129,1,0.0260,...,0.1720,0.3170,74.807,audio_features,3SVAN3BRByDmHOhKyIDxfC,spotify:track:3SVAN3BRByDmHOhKyIDxfC,https://api.spotify.com/v1/tracks/3SVAN3BRByDm...,https://api.spotify.com/v1/audio-analysis/3SVA...,264067,4
5,5,Just Breathe,6i81qFkru6Kj1IEsB7KNp2,Pearl Jam,0.473,0.348,0,-11.100,1,0.0288,...,0.0823,0.2830,107.819,audio_features,6i81qFkru6Kj1IEsB7KNp2,spotify:track:6i81qFkru6Kj1IEsB7KNp2,https://api.spotify.com/v1/tracks/6i81qFkru6Kj...,https://api.spotify.com/v1/audio-analysis/6i81...,215853,4
6,6,Disarm - Remastered,7A8caPxvrT8ZgLvQPPUCcj,The Smashing Pumpkins,0.527,0.418,4,-10.205,0,0.0277,...,0.1390,0.2170,129.517,audio_features,7A8caPxvrT8ZgLvQPPUCcj,spotify:track:7A8caPxvrT8ZgLvQPPUCcj,https://api.spotify.com/v1/tracks/7A8caPxvrT8Z...,https://api.spotify.com/v1/audio-analysis/7A8c...,196905,4
7,7,If I Had A Gun,0Pk9Lc2RFHVffFgjozPIxp,Noel Gallagher's High Flying Birds,0.337,0.736,6,-3.141,0,0.0334,...,0.1800,0.2130,76.370,audio_features,0Pk9Lc2RFHVffFgjozPIxp,spotify:track:0Pk9Lc2RFHVffFgjozPIxp,https://api.spotify.com/v1/tracks/0Pk9Lc2RFHVf...,https://api.spotify.com/v1/audio-analysis/0Pk9...,249107,4
8,8,One,3PKtemUKxiDBvBo7tpQ8bG,U2,0.511,0.536,0,-8.661,1,0.0302,...,0.1220,0.3570,90.358,audio_features,3PKtemUKxiDBvBo7tpQ8bG,spotify:track:3PKtemUKxiDBvBo7tpQ8bG,https://api.spotify.com/v1/tracks/3PKtemUKxiDB...,https://api.spotify.com/v1/audio-analysis/3PKt...,276187,4
9,9,Ain't It The Life,2ifYUfvMNbXQDPJEyk68rP,Foo Fighters,0.570,0.836,0,-4.580,1,0.0258,...,0.3290,0.3030,108.866,audio_features,2ifYUfvMNbXQDPJEyk68rP,spotify:track:2ifYUfvMNbXQDPJEyk68rP,https://api.spotify.com/v1/tracks/2ifYUfvMNbXQ...,https://api.spotify.com/v1/audio-analysis/2ifY...,254773,4


### Soulful Blend

In [28]:
fifth_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DX8Md3JnnrexB")

In [29]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
fifth_results = get_tracks("Spotify", "37i9dQZF1DX8Md3JnnrexB")

In [30]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = fifth_results
for r in fifth_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist5_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [31]:
features5_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:]))

In [32]:
fifth_playlist = pd.concat([playlist5_info, features5_info], axis=1)
fifth_playlist.reset_index()

,index,song_name,song_id,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,Nightshift,2jGfOCUOXBxrV63jlFVZMV,Commodores,0.727,0.585,8,-8.996,1,0.0333,...,0.0565,0.428,105.061,audio_features,2jGfOCUOXBxrV63jlFVZMV,spotify:track:2jGfOCUOXBxrV63jlFVZMV,https://api.spotify.com/v1/tracks/2jGfOCUOXBxr...,https://api.spotify.com/v1/audio-analysis/2jGf...,304027,4
1,1,Until You Come Back to Me (That's What I'm Gon...,3iX0BMs9dX9CsgrHhiB173,Aretha Franklin,0.664,0.634,2,-6.348,1,0.0349,...,0.2210,0.666,93.914,audio_features,3iX0BMs9dX9CsgrHhiB173,spotify:track:3iX0BMs9dX9CsgrHhiB173,https://api.spotify.com/v1/tracks/3iX0BMs9dX9C...,https://api.spotify.com/v1/audio-analysis/3iX0...,208467,4
2,2,On My Own,7KsXSnQc8qdeBSOnTM5Vvn,Patti LaBelle,0.615,0.455,5,-11.050,1,0.0262,...,0.4150,0.483,92.163,audio_features,7KsXSnQc8qdeBSOnTM5Vvn,spotify:track:7KsXSnQc8qdeBSOnTM5Vvn,https://api.spotify.com/v1/tracks/7KsXSnQc8qde...,https://api.spotify.com/v1/audio-analysis/7KsX...,289173,4
3,3,New York Eyes,20oUeycXhRS3iNp0Hl32hh,Nicole,0.669,0.716,2,-7.292,0,0.0523,...,0.0507,0.961,79.475,audio_features,20oUeycXhRS3iNp0Hl32hh,spotify:track:20oUeycXhRS3iNp0Hl32hh,https://api.spotify.com/v1/tracks/20oUeycXhRS3...,https://api.spotify.com/v1/audio-analysis/20oU...,283053,4
4,4,Let's Stay Together,7kWhdmRYv8CqbWNqfojqVd,Al Green,0.746,0.273,5,-16.630,1,0.0573,...,0.0526,0.502,101.952,audio_features,7kWhdmRYv8CqbWNqfojqVd,spotify:track:7kWhdmRYv8CqbWNqfojqVd,https://api.spotify.com/v1/tracks/7kWhdmRYv8Cq...,https://api.spotify.com/v1/audio-analysis/7kWh...,199400,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,You Make Me Feel Brand New,00CCi9yHJh7NjXcNT565Ja,The Stylistics,0.451,0.359,7,-8.567,1,0.0247,...,0.1370,0.178,76.015,audio_features,00CCi9yHJh7NjXcNT565Ja,spotify:track:00CCi9yHJh7NjXcNT565Ja,https://api.spotify.com/v1/tracks/00CCi9yHJh7N...,https://api.spotify.com/v1/audio-analysis/00CC...,288440,4
96,96,Grow,3jPd7hIM0feCje8ZbVu6en,Samm Henshaw,0.672,0.672,9,-7.693,1,0.0847,...,0.0823,0.807,85.012,audio_features,3jPd7hIM0feCje8ZbVu6en,spotify:track:3jPd7hIM0feCje8ZbVu6en,https://api.spotify.com/v1/tracks/3jPd7hIM0feC...,https://api.spotify.com/v1/audio-analysis/3jPd...,180523,4
97,97,Be Here in Te Morning,3uaCC1kjEA6PmDcCR23Iha,Joy Denalane,0.480,0.732,0,-5.494,1,0.0433,...,0.7990,0.518,135.790,audio_features,3uaCC1kjEA6PmDcCR23Iha,spotify:track:3uaCC1kjEA6PmDcCR23Iha,https://api.spotify.com/v1/tracks/3uaCC1kjEA6P...,https://api.spotify.com/v1/audio-analysis/3uaC...,244640,4
98,98,Tired of Being Alone,5vjmFhes2UVEo5aGQ27q7l,Al Green,0.769,0.390,7,-8.525,1,0.0391,...,0.0673,0.610,98.077,audio_features,5vjmFhes2UVEo5aGQ27q7l,spotify:track:5vjmFhes2UVEo5aGQ27q7l,https://api.spotify.com/v1/tracks/5vjmFhes2UVE...,https://api.spotify.com/v1/audio-analysis/5vjm...,172320,4


### Morning Coffee

In [33]:
sixth_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DXcgZcN2HVMoe")

In [34]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
sixth_results = get_tracks("Spotify", "37i9dQZF1DXcgZcN2HVMoe")

In [35]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = sixth_results
for r in sixth_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist6_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [36]:
features6_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features6_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features6_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[199:]))
features6 = pd.concat([features6_info, features6_info2, features6_info3])
features6.reset_index(drop=True, inplace=True)

In [37]:
sixth_playlist = pd.concat([playlist6_info, features6], axis=1)
sixth_playlist.reset_index()

,index,song_name,song_id,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,In a Stranger's Arms,5Tl8bvgo5shLODXiT9ckva,LÉON,0.517,0.435,9,-7.532,1,0.0278,...,0.1230,0.335,97.931,audio_features,5Tl8bvgo5shLODXiT9ckva,spotify:track:5Tl8bvgo5shLODXiT9ckva,https://api.spotify.com/v1/tracks/5Tl8bvgo5shL...,https://api.spotify.com/v1/audio-analysis/5Tl8...,200220,4
1,1,Dear August,16gW9O8A4rIMx5OOVlzsSn,PJ Harding,0.643,0.324,0,-7.817,1,0.0338,...,0.1110,0.444,147.019,audio_features,16gW9O8A4rIMx5OOVlzsSn,spotify:track:16gW9O8A4rIMx5OOVlzsSn,https://api.spotify.com/v1/tracks/16gW9O8A4rIM...,https://api.spotify.com/v1/audio-analysis/16gW...,200940,4
2,2,look up,1Thyh3NBnJ783exapZJIA3,Joy Oladokun,0.376,0.290,0,-10.157,1,0.0303,...,0.1410,0.223,90.349,audio_features,1Thyh3NBnJ783exapZJIA3,spotify:track:1Thyh3NBnJ783exapZJIA3,https://api.spotify.com/v1/tracks/1Thyh3NBnJ78...,https://api.spotify.com/v1/audio-analysis/1Thy...,213938,4
3,3,Rockland,5V03tsbEzYG0PmRrh3K4rF,Gracie Abrams,0.568,0.402,5,-11.344,1,0.0448,...,0.1030,0.397,125.743,audio_features,5V03tsbEzYG0PmRrh3K4rF,spotify:track:5V03tsbEzYG0PmRrh3K4rF,https://api.spotify.com/v1/tracks/5V03tsbEzYG0...,https://api.spotify.com/v1/audio-analysis/5V03...,217369,4
4,4,Amsterdam,1sIAk1Hpj5YsMyYYtTsdal,Wild Rivers,0.637,0.484,1,-9.426,1,0.0284,...,0.0904,0.590,140.088,audio_features,1sIAk1Hpj5YsMyYYtTsdal,spotify:track:1sIAk1Hpj5YsMyYYtTsdal,https://api.spotify.com/v1/tracks/1sIAk1Hpj5Ys...,https://api.spotify.com/v1/audio-analysis/1sIA...,213920,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,245,The Wire - Alternate Version,0ZwGSa87JHR86PJa3Fzgru,Patrick Droney,0.500,0.299,0,-7.767,1,0.0478,...,0.0866,0.340,96.017,audio_features,0ZwGSa87JHR86PJa3Fzgru,spotify:track:0ZwGSa87JHR86PJa3Fzgru,https://api.spotify.com/v1/tracks/0ZwGSa87JHR8...,https://api.spotify.com/v1/audio-analysis/0ZwG...,221063,4
246,246,Friendly Fire,1u71gNzUwTjgfYmArCUD0m,Holly Humberstone,0.465,0.602,10,-9.664,1,0.0780,...,0.1010,0.595,159.905,audio_features,1u71gNzUwTjgfYmArCUD0m,spotify:track:1u71gNzUwTjgfYmArCUD0m,https://api.spotify.com/v1/tracks/1u71gNzUwTjg...,https://api.spotify.com/v1/audio-analysis/1u71...,222800,4
247,247,Thinking of Me,7gXTJiXy2V4wC3BIUKJOsB,Cole Scheifele,0.472,0.419,2,-10.134,1,0.0382,...,0.2860,0.428,145.161,audio_features,7gXTJiXy2V4wC3BIUKJOsB,spotify:track:7gXTJiXy2V4wC3BIUKJOsB,https://api.spotify.com/v1/tracks/7gXTJiXy2V4w...,https://api.spotify.com/v1/audio-analysis/7gXT...,188627,3
248,248,Sleepless in Seattle,1JEjCzwE9viGiRyewkcWy9,Noah Gundersen,0.378,0.195,11,-12.423,1,0.0300,...,0.1010,0.356,93.071,audio_features,1JEjCzwE9viGiRyewkcWy9,spotify:track:1JEjCzwE9viGiRyewkcWy9,https://api.spotify.com/v1/tracks/1JEjCzwE9viG...,https://api.spotify.com/v1/audio-analysis/1JEj...,240813,4


### 500 best songs of all times

In [38]:
seventh_playlist = sp.user_playlist_tracks("Arian", "5dxn0i8MPl6XFVVxNatd6U")

In [39]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
seventh_results = get_tracks("Arian", "5dxn0i8MPl6XFVVxNatd6U")

In [40]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = seventh_results
for r in seventh_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist7_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [41]:
features7_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features7_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features7_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[199:299]))
features7_info4 = pd.DataFrame(sp.audio_features(tracks = song_id[299:399]))
features7_info5 = pd.DataFrame(sp.audio_features(tracks = song_id[399:499]))
features7_info6 = pd.DataFrame(sp.audio_features(tracks = song_id[499:]))
features7 = pd.concat([features7_info, features7_info2, features7_info3, features7_info4, features7_info5, features7_info6])
features7.reset_index(drop=True, inplace=True)

In [42]:
seventh_playlist = pd.concat([playlist7_info, features7], axis=1)
seventh_playlist.reset_index()

,index,song_name,song_id,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,Respect,7s25THrKz86DM225dOYwnr,Aretha Franklin,0.805,0.558,0,-5.226,1,0.0410,...,0.0546,0.965,114.950,audio_features,7s25THrKz86DM225dOYwnr,spotify:track:7s25THrKz86DM225dOYwnr,https://api.spotify.com/v1/tracks/7s25THrKz86D...,https://api.spotify.com/v1/audio-analysis/7s25...,147600,4
1,1,Fight The Power,1yo16b3u0lptm6Cs7lx4AD,Public Enemy,0.797,0.582,2,-12.970,1,0.2550,...,0.5170,0.415,105.974,audio_features,1yo16b3u0lptm6Cs7lx4AD,spotify:track:1yo16b3u0lptm6Cs7lx4AD,https://api.spotify.com/v1/tracks/1yo16b3u0lpt...,https://api.spotify.com/v1/audio-analysis/1yo1...,282640,4
2,2,A Change Is Gonna Come,0KOE1hat4SIer491XKk4Pa,Sam Cooke,0.212,0.383,10,-10.071,1,0.0347,...,0.2990,0.452,173.790,audio_features,0KOE1hat4SIer491XKk4Pa,spotify:track:0KOE1hat4SIer491XKk4Pa,https://api.spotify.com/v1/tracks/0KOE1hat4SIe...,https://api.spotify.com/v1/audio-analysis/0KOE...,191160,3
3,3,Like a Rolling Stone,3AhXZa8sUQht0UEdBJgpGc,Bob Dylan,0.482,0.721,0,-6.839,1,0.0321,...,0.1890,0.557,95.263,audio_features,3AhXZa8sUQht0UEdBJgpGc,spotify:track:3AhXZa8sUQht0UEdBJgpGc,https://api.spotify.com/v1/tracks/3AhXZa8sUQht...,https://api.spotify.com/v1/audio-analysis/3AhX...,369600,4
4,4,Smells Like Teen Spirit,5ghIJDpPoe3CfHMGu71E6T,Nirvana,0.502,0.912,1,-4.556,1,0.0564,...,0.1060,0.720,116.761,audio_features,5ghIJDpPoe3CfHMGu71E6T,spotify:track:5ghIJDpPoe3CfHMGu71E6T,https://api.spotify.com/v1/tracks/5ghIJDpPoe3C...,https://api.spotify.com/v1/audio-analysis/5ghI...,301920,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,Without You,6MrILERJLA3Jcq0E9poZsp,Harry Nilsson,0.381,0.186,4,-15.477,1,0.0290,...,0.0694,0.142,65.058,audio_features,6MrILERJLA3Jcq0E9poZsp,spotify:track:6MrILERJLA3Jcq0E9poZsp,https://api.spotify.com/v1/tracks/6MrILERJLA3J...,https://api.spotify.com/v1/audio-analysis/6MrI...,201000,4
496,496,Truth Hurts,3HWzoMvoF3TQfYg4UPszDq,Lizzo,0.715,0.624,4,-3.046,0,0.1140,...,0.1230,0.412,158.087,audio_features,3HWzoMvoF3TQfYg4UPszDq,spotify:track:3HWzoMvoF3TQfYg4UPszDq,https://api.spotify.com/v1/tracks/3HWzoMvoF3TQ...,https://api.spotify.com/v1/audio-analysis/3HWz...,173325,4
497,497,Pancho and Lefty,42Wb5T7YmdrzVD81cUWWAg,Townes Van Zandt,0.257,0.453,2,-8.063,1,0.0305,...,0.5640,0.397,148.848,audio_features,42Wb5T7YmdrzVD81cUWWAg,spotify:track:42Wb5T7YmdrzVD81cUWWAg,https://api.spotify.com/v1/tracks/42Wb5T7Ymdrz...,https://api.spotify.com/v1/audio-analysis/42Wb...,241080,4
498,498,Baby Love,5uES1C2NgkdrNHiCwf9jRr,The Supremes,0.595,0.643,5,-9.275,1,0.0365,...,0.5380,0.730,135.633,audio_features,5uES1C2NgkdrNHiCwf9jRr,spotify:track:5uES1C2NgkdrNHiCwf9jRr,https://api.spotify.com/v1/tracks/5uES1C2Ngkdr...,https://api.spotify.com/v1/audio-analysis/5uES...,158040,4


### Best fo the Oldies

In [43]:
eighth_playlist = sp.user_playlist_tracks("Marcel", "6OAF6gkxAe4lqHdSLmGpdd")

In [44]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
eighth_results = get_tracks("Marcel", "6OAF6gkxAe4lqHdSLmGpdd")

In [45]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = eighth_results
for r in eighth_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist8_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [46]:
features8_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features8_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features8_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[199:299]))
features8_info4 = pd.DataFrame(sp.audio_features(tracks = song_id[299:399]))
features8_info5 = pd.DataFrame(sp.audio_features(tracks = song_id[399:499]))
features8_info6 = pd.DataFrame(sp.audio_features(tracks = song_id[499:599]))
features8_info7 = pd.DataFrame(sp.audio_features(tracks = song_id[599:699]))
features8_info8 = pd.DataFrame(sp.audio_features(tracks = song_id[699:799]))
features8_info9 = pd.DataFrame(sp.audio_features(tracks = song_id[799:899]))
features8_info10 = pd.DataFrame(sp.audio_features(tracks = song_id[899:999]))
features8_info11 = pd.DataFrame(sp.audio_features(tracks = song_id[999:]))
features8 = pd.concat([features8_info, features8_info2, features8_info3, features8_info4, features8_info5, features8_info6, features8_info7, features8_info8, features8_info9, features8_info10, features8_info11])
features8.reset_index(drop=True, inplace=True)

In [47]:
eighth_playlist = pd.concat([playlist8_info, features8], axis=1)
eighth_playlist.reset_index()

,index,song_name,song_id,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,I'm Still Standing,5fwP61Nmo5xXYIjO9XEWqb,Elton John,0.504,0.904,6,-6.863,1,0.1790,...,0.1400,0.772,176.808,audio_features,5fwP61Nmo5xXYIjO9XEWqb,spotify:track:5fwP61Nmo5xXYIjO9XEWqb,https://api.spotify.com/v1/tracks/5fwP61Nmo5xX...,https://api.spotify.com/v1/audio-analysis/5fwP...,183440,4
1,1,Le Freak,28NBmftocOzTPEb6OYA9fW,CHIC,0.867,0.746,7,-7.445,1,0.0475,...,0.1230,0.933,118.616,audio_features,28NBmftocOzTPEb6OYA9fW,spotify:track:28NBmftocOzTPEb6OYA9fW,https://api.spotify.com/v1/tracks/28NBmftocOzT...,https://api.spotify.com/v1/audio-analysis/28NB...,327573,4
2,2,I Feel Love,6efRrfjV1YW1rtecLUHoP1,Donna Summer,0.690,0.675,0,-16.247,1,0.0433,...,0.0987,0.940,128.100,audio_features,6efRrfjV1YW1rtecLUHoP1,spotify:track:6efRrfjV1YW1rtecLUHoP1,https://api.spotify.com/v1/tracks/6efRrfjV1YW1...,https://api.spotify.com/v1/audio-analysis/6efR...,493600,4
3,3,Blueprint,1RuzwWdidBXXMVpwoBBIL1,Rainbirds,0.475,0.757,0,-12.237,0,0.0451,...,0.1470,0.715,194.193,audio_features,1RuzwWdidBXXMVpwoBBIL1,spotify:track:1RuzwWdidBXXMVpwoBBIL1,https://api.spotify.com/v1/tracks/1RuzwWdidBXX...,https://api.spotify.com/v1/audio-analysis/1Ruz...,331467,4
4,4,Life Is For Living,6OAVhdZQs1KMNH1TfXbFJE,Barclay James Harvest,0.700,0.565,7,-14.761,1,0.0823,...,0.0724,0.919,103.109,audio_features,6OAVhdZQs1KMNH1TfXbFJE,spotify:track:6OAVhdZQs1KMNH1TfXbFJE,https://api.spotify.com/v1/tracks/6OAVhdZQs1KM...,https://api.spotify.com/v1/audio-analysis/6OAV...,218933,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,1044,Enola Gay - Remastered,3NYCaxkggl0Hh8vQptSUvV,Orchestral Manoeuvres In The Dark,0.559,0.764,2,-7.058,0,0.0348,...,0.0807,0.932,143.319,audio_features,3NYCaxkggl0Hh8vQptSUvV,spotify:track:3NYCaxkggl0Hh8vQptSUvV,https://api.spotify.com/v1/tracks/3NYCaxkggl0H...,https://api.spotify.com/v1/audio-analysis/3NYC...,213693,4
1045,1045,Dancing in the City,4MVhYm4LCenxBIZKrT5CAc,Marshall Hain,0.732,0.431,0,-13.946,1,0.0504,...,0.1020,0.564,101.803,audio_features,4MVhYm4LCenxBIZKrT5CAc,spotify:track:4MVhYm4LCenxBIZKrT5CAc,https://api.spotify.com/v1/tracks/4MVhYm4LCenx...,https://api.spotify.com/v1/audio-analysis/4MVh...,226867,4
1046,1046,So You Win Again,1tMi57U5Nkk6qRZSNwbzQr,Hot Chocolate,0.706,0.538,9,-9.188,1,0.0406,...,0.3880,0.816,104.900,audio_features,1tMi57U5Nkk6qRZSNwbzQr,spotify:track:1tMi57U5Nkk6qRZSNwbzQr,https://api.spotify.com/v1/tracks/1tMi57U5Nkk6...,https://api.spotify.com/v1/audio-analysis/1tMi...,271427,4
1047,1047,You See The Trouble With Me,3O48gtvD20jxiNtT7tWr4C,Barry White,0.666,0.583,6,-13.698,0,0.0284,...,0.0772,0.746,114.851,audio_features,3O48gtvD20jxiNtT7tWr4C,spotify:track:3O48gtvD20jxiNtT7tWr4C,https://api.spotify.com/v1/tracks/3O48gtvD20jx...,https://api.spotify.com/v1/audio-analysis/3O48...,209907,4


### 1001 songs you need to hear before you die

In [48]:
ninth_playlist = sp.user_playlist_tracks("Jimmy Alexander", "6aCHA5HeRfz5gJjllGZ7Of")

In [49]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
ninth_results = get_tracks("Jimmy Alexander", "6aCHA5HeRfz5gJjllGZ7Of")

In [50]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = ninth_results
for r in ninth_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist9_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [51]:
features9_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features9_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features9_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[199:299]))
features9_info4 = pd.DataFrame(sp.audio_features(tracks = song_id[299:399]))
features9_info5 = pd.DataFrame(sp.audio_features(tracks = song_id[399:499]))
features9_info6 = pd.DataFrame(sp.audio_features(tracks = song_id[499:599]))
features9_info7 = pd.DataFrame(sp.audio_features(tracks = song_id[599:699]))
features9_info8 = pd.DataFrame(sp.audio_features(tracks = song_id[699:799]))
features9_info9 = pd.DataFrame(sp.audio_features(tracks = song_id[799:899]))
features9_info10 = pd.DataFrame(sp.audio_features(tracks = song_id[899:999]))
features9_info11 = pd.DataFrame(sp.audio_features(tracks = song_id[999:]))
features9 = pd.concat([features9_info, features9_info2, features9_info3, features9_info4, features9_info5, features9_info6, features9_info7, features9_info8, features9_info9, features9_info10, features9_info11])
features9.reset_index(drop=True, inplace=True)

In [52]:
ninth_playlist = pd.concat([playlist9_info, features9], axis=1)
ninth_playlist.reset_index()

,index,song_name,song_id,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,'O Sole Mio,0oRa6dxPCzotYzujp21QFK,Enrico Caruso,0.287,0.149,7,-13.503,1,0.0379,...,0.2060,0.269,129.377,audio_features,0oRa6dxPCzotYzujp21QFK,spotify:track:0oRa6dxPCzotYzujp21QFK,https://api.spotify.com/v1/tracks/0oRa6dxPCzot...,https://api.spotify.com/v1/audio-analysis/0oRa...,204293,4
1,1,St. Louis Blues (with Louis Armstrong),4ejUoPIMggFTVrAyvodqJQ,Bessie Smith,0.408,0.145,10,-10.990,1,0.0501,...,0.1380,0.294,68.800,audio_features,4ejUoPIMggFTVrAyvodqJQ,spotify:track:4ejUoPIMggFTVrAyvodqJQ,https://api.spotify.com/v1/tracks/4ejUoPIMggFT...,https://api.spotify.com/v1/audio-analysis/4ejU...,190107,4
2,2,Allons A Lafayette,3zZyFcJZ84mjQWLNrtMr2Z,Joe Falcon & Cleoma Breaux,0.533,0.405,7,-14.048,1,0.0377,...,0.3930,0.868,93.960,audio_features,3zZyFcJZ84mjQWLNrtMr2Z,spotify:track:3zZyFcJZ84mjQWLNrtMr2Z,https://api.spotify.com/v1/tracks/3zZyFcJZ84mj...,https://api.spotify.com/v1/audio-analysis/3zZy...,176493,4
3,3,Lágrimas Negras,5UmrMwj8Ouw6yeg3TxQ9GA,Trio Matamoros,0.613,0.305,0,-10.041,0,0.2190,...,0.1110,0.789,96.967,audio_features,5UmrMwj8Ouw6yeg3TxQ9GA,spotify:track:5UmrMwj8Ouw6yeg3TxQ9GA,https://api.spotify.com/v1/tracks/5UmrMwj8Ouw6...,https://api.spotify.com/v1/audio-analysis/5Umr...,174520,4
4,4,St. James Infirmary,7fAa9rz4UmwuB4AGh50Gmp,Louis Armstrong & His Savoy Ballroom Five,0.693,0.182,5,-11.022,0,0.0514,...,0.1530,0.588,116.508,audio_features,7fAa9rz4UmwuB4AGh50Gmp,spotify:track:7fAa9rz4UmwuB4AGh50Gmp,https://api.spotify.com/v1/tracks/7fAa9rz4Umwu...,https://api.spotify.com/v1/audio-analysis/7fAa...,191867,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027,1027,Tilted,4U6GYrZi0xvNkUmFitCDED,Christine and the Queens,0.872,0.361,7,-9.880,1,0.0609,...,0.1080,0.489,100.001,audio_features,4U6GYrZi0xvNkUmFitCDED,spotify:track:4U6GYrZi0xvNkUmFitCDED,https://api.spotify.com/v1/tracks/4U6GYrZi0xvN...,https://api.spotify.com/v1/audio-analysis/4U6G...,233247,4
1028,1028,Million Reasons,7dZ1Odmx9jWIweQSatnRqo,Lady Gaga,0.666,0.423,0,-8.012,1,0.0430,...,0.1060,0.154,129.890,audio_features,7dZ1Odmx9jWIweQSatnRqo,spotify:track:7dZ1Odmx9jWIweQSatnRqo,https://api.spotify.com/v1/tracks/7dZ1Odmx9jWI...,https://api.spotify.com/v1/audio-analysis/7dZ1...,205280,4
1029,1029,Lord,09CViXNGyH05AItXMtM6fC,Young Fathers,0.226,0.328,10,-9.804,1,0.0330,...,0.0851,0.119,175.558,audio_features,09CViXNGyH05AItXMtM6fC,spotify:track:09CViXNGyH05AItXMtM6fC,https://api.spotify.com/v1/tracks/09CViXNGyH05...,https://api.spotify.com/v1/audio-analysis/09CV...,217120,3
1030,1030,STREET LIVIN',6NjcOqkCcsbO65KhM5Af2K,Black Eyed Peas,0.693,0.730,4,-4.270,0,0.2210,...,0.3820,0.592,99.633,audio_features,6NjcOqkCcsbO65KhM5Af2K,spotify:track:6NjcOqkCcsbO65KhM5Af2K,https://api.spotify.com/v1/tracks/6NjcOqkCcsbO...,https://api.spotify.com/v1/audio-analysis/6Njc...,185413,4


### The ultimate huge playlist

In [53]:
tenth_playlist = sp.user_playlist_tracks("AROL", "20JT4SnBRiiXBeIXr5f2c0")

In [54]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
tenth_results = get_tracks("AROL", "20JT4SnBRiiXBeIXr5f2c0")

In [55]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = tenth_results
for r in tenth_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist10_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [56]:
features10_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features10_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features10_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[199:299]))
features10_info4 = pd.DataFrame(sp.audio_features(tracks = song_id[299:399]))
features10_info5 = pd.DataFrame(sp.audio_features(tracks = song_id[399:499]))
features10_info6 = pd.DataFrame(sp.audio_features(tracks = song_id[499:599]))
features10_info7 = pd.DataFrame(sp.audio_features(tracks = song_id[599:699]))
features10_info8 = pd.DataFrame(sp.audio_features(tracks = song_id[699:799]))
features10_info9 = pd.DataFrame(sp.audio_features(tracks = song_id[799:899]))
features10_info10 = pd.DataFrame(sp.audio_features(tracks = song_id[899:999]))
features10_info11 = pd.DataFrame(sp.audio_features(tracks = song_id[999:1099]))
features10_info12 = pd.DataFrame(sp.audio_features(tracks = song_id[1099:1199]))
features10_info13 = pd.DataFrame(sp.audio_features(tracks = song_id[1199:1299]))
features10_info14 = pd.DataFrame(sp.audio_features(tracks = song_id[1299:1399]))
features10_info15 = pd.DataFrame(sp.audio_features(tracks = song_id[1399:1499]))
features10_info16 = pd.DataFrame(sp.audio_features(tracks = song_id[1499:1599]))
features10_info17 = pd.DataFrame(sp.audio_features(tracks = song_id[1599:1699]))
features10_info18 = pd.DataFrame(sp.audio_features(tracks = song_id[1699:1799]))
features10_info19 = pd.DataFrame(sp.audio_features(tracks = song_id[1799:1899]))
features10_info20 = pd.DataFrame(sp.audio_features(tracks = song_id[1899:1999]))
features10_info21 = pd.DataFrame(sp.audio_features(tracks = song_id[1999:2099]))
features10_info22 = pd.DataFrame(sp.audio_features(tracks = song_id[2099:2199]))
features10_info23 = pd.DataFrame(sp.audio_features(tracks = song_id[2199:2299]))
features10_info24 = pd.DataFrame(sp.audio_features(tracks = song_id[2299:2399]))
features10_info25 = pd.DataFrame(sp.audio_features(tracks = song_id[2399:2499]))
features10_info26 = pd.DataFrame(sp.audio_features(tracks = song_id[2499:2599]))
features10_info27 = pd.DataFrame(sp.audio_features(tracks = song_id[2599:2699]))
features10_info28 = pd.DataFrame(sp.audio_features(tracks = song_id[2699:2799]))
features10_info29 = pd.DataFrame(sp.audio_features(tracks = song_id[2799:2899]))
features10_info30 = pd.DataFrame(sp.audio_features(tracks = song_id[2899:2999]))
features10_info31 = pd.DataFrame(sp.audio_features(tracks = song_id[2999:3099]))
features10_info32 = pd.DataFrame(sp.audio_features(tracks = song_id[3099:3199]))
features10_info33 = pd.DataFrame(sp.audio_features(tracks = song_id[3199:3299]))
features10_info34 = pd.DataFrame(sp.audio_features(tracks = song_id[3299:3399]))
features10_info35 = pd.DataFrame(sp.audio_features(tracks = song_id[3399:3499]))
features10_info36 = pd.DataFrame(sp.audio_features(tracks = song_id[3499:3599]))
features10_info37 = pd.DataFrame(sp.audio_features(tracks = song_id[3599:3699]))
features10_info38 = pd.DataFrame(sp.audio_features(tracks = song_id[3699:3799]))
features10_info39 = pd.DataFrame(sp.audio_features(tracks = song_id[3799:3899]))
features10_info40 = pd.DataFrame(sp.audio_features(tracks = song_id[3899:3999]))
features10_info41 = pd.DataFrame(sp.audio_features(tracks = song_id[3999:4099]))
features10_info42 = pd.DataFrame(sp.audio_features(tracks = song_id[4099:4199]))
features10_info43 = pd.DataFrame(sp.audio_features(tracks = song_id[4199:4299]))
features10_info44 = pd.DataFrame(sp.audio_features(tracks = song_id[4299:4399]))
features10_info45 = pd.DataFrame(sp.audio_features(tracks = song_id[4399:4499]))
features10_info46 = pd.DataFrame(sp.audio_features(tracks = song_id[4499:4599]))
features10_info47 = pd.DataFrame(sp.audio_features(tracks = song_id[4599:]))
features10 = pd.concat([features9_info, features10_info2, features10_info3, features10_info4, 
                        features10_info5, features10_info6, features10_info7, features10_info8, 
                        features10_info9, features10_info10, features10_info11, features10_info12, 
                        features10_info13, features10_info14, features10_info15, features10_info16, 
                        features10_info17, features10_info18, features10_info19, features10_info20, 
                        features10_info21, features10_info22, features10_info23, features10_info24, 
                        features10_info25, features10_info26, features10_info27, features10_info28, 
                        features10_info29, features10_info30, features10_info31, features10_info32, 
                        features10_info33, features10_info34, features10_info35, features10_info36, 
                        features10_info37, features10_info38, features10_info39, features10_info40, 
                        features10_info41, features10_info42, features10_info43, features10_info44, 
                        features10_info45, features10_info46, features10_info47])
features10.reset_index(drop=True, inplace=True)

In [57]:
tenth_playlist = pd.concat([playlist10_info, features10], axis=1)
tenth_playlist.reset_index()

,index,song_name,song_id,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,Thunderstruck,57bgtoPSgt236HzfBOd8kj,AC/DC,0.287,0.149,7,-13.503,1,0.0379,...,0.2060,0.269,129.377,audio_features,0oRa6dxPCzotYzujp21QFK,spotify:track:0oRa6dxPCzotYzujp21QFK,https://api.spotify.com/v1/tracks/0oRa6dxPCzot...,https://api.spotify.com/v1/audio-analysis/0oRa...,204293,4
1,1,Shoot to Thrill,0C80GCp0mMuBzLf3EAXqxv,AC/DC,0.408,0.145,10,-10.990,1,0.0501,...,0.1380,0.294,68.800,audio_features,4ejUoPIMggFTVrAyvodqJQ,spotify:track:4ejUoPIMggFTVrAyvodqJQ,https://api.spotify.com/v1/tracks/4ejUoPIMggFT...,https://api.spotify.com/v1/audio-analysis/4ejU...,190107,4
2,2,Left Hand Free,126TblwXGNTUZ7RPMnThkU,alt-J,0.533,0.405,7,-14.048,1,0.0377,...,0.3930,0.868,93.960,audio_features,3zZyFcJZ84mjQWLNrtMr2Z,spotify:track:3zZyFcJZ84mjQWLNrtMr2Z,https://api.spotify.com/v1/tracks/3zZyFcJZ84mj...,https://api.spotify.com/v1/audio-analysis/3zZy...,176493,4
3,3,Why'd You Only Call Me When You're High?,086myS9r57YsLbJpU0TgK9,Arctic Monkeys,0.613,0.305,0,-10.041,0,0.2190,...,0.1110,0.789,96.967,audio_features,5UmrMwj8Ouw6yeg3TxQ9GA,spotify:track:5UmrMwj8Ouw6yeg3TxQ9GA,https://api.spotify.com/v1/tracks/5UmrMwj8Ouw6...,https://api.spotify.com/v1/audio-analysis/5Umr...,174520,4
4,4,I Bet You Look Good On The Dancefloor,3DQVgcqaP3iSMbaKsd57l5,Arctic Monkeys,0.693,0.182,5,-11.022,0,0.0514,...,0.1530,0.588,116.508,audio_features,7fAa9rz4UmwuB4AGh50Gmp,spotify:track:7fAa9rz4UmwuB4AGh50Gmp,https://api.spotify.com/v1/tracks/7fAa9rz4Umwu...,https://api.spotify.com/v1/audio-analysis/7fAa...,191867,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4693,4693,Father and Daughter,4aU1zq0W9Ftg1UAQSj4aHN,Paul Simon,0.775,0.560,4,-7.085,1,0.0314,...,0.1830,0.811,106.003,audio_features,4aU1zq0W9Ftg1UAQSj4aHN,spotify:track:4aU1zq0W9Ftg1UAQSj4aHN,https://api.spotify.com/v1/tracks/4aU1zq0W9Ftg...,https://api.spotify.com/v1/audio-analysis/4aU1...,251187,4
4694,4694,Daughters,5FPnjikbwlDMULCCCa6ZCJ,John Mayer,0.674,0.413,2,-7.816,1,0.0274,...,0.0980,0.503,124.893,audio_features,5FPnjikbwlDMULCCCa6ZCJ,spotify:track:5FPnjikbwlDMULCCCa6ZCJ,https://api.spotify.com/v1/tracks/5FPnjikbwlDM...,https://api.spotify.com/v1/audio-analysis/5FPn...,237907,3
4695,4695,"Mahna, Mahna",5FnyVsDPOblZ8gVvaXta9z,CAKE,0.742,0.483,7,-11.369,1,0.1550,...,0.0733,0.638,89.883,audio_features,5FnyVsDPOblZ8gVvaXta9z,spotify:track:5FnyVsDPOblZ8gVvaXta9z,https://api.spotify.com/v1/tracks/5FnyVsDPOblZ...,https://api.spotify.com/v1/audio-analysis/5Fny...,174067,4
4696,4696,Child of Mine,4A7oANUhxquUpTCuu6GmoD,Carole King,0.492,0.117,0,-17.819,1,0.0387,...,0.0812,0.332,120.839,audio_features,4A7oANUhxquUpTCuu6GmoD,spotify:track:4A7oANUhxquUpTCuu6GmoD,https://api.spotify.com/v1/tracks/4A7oANUhxquU...,https://api.spotify.com/v1/audio-analysis/4A7o...,242467,4


### Creating the big dataframe

In [58]:
huge_playlist = pd.concat([first_playlist, second_playlist, third_playlist, fourth_playlist, 
                           fifth_playlist, sixth_playlist, seventh_playlist, eighth_playlist, 
                           ninth_playlist, tenth_playlist])
huge_playlist.reset_index(drop=True, inplace=True)

In [59]:
# count duplicated values and dropping them
huge_playlist.duplicated(subset='song_name', keep='first').sum()
huge_playlist = huge_playlist.drop_duplicates()
huge_playlist.reset_index(drop=True, inplace=True)
huge_playlist = huge_playlist.dropna()
huge_playlist.reset_index(drop=True, inplace=True)

## Clustering

In [60]:
# even MORE packages!
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import plotly 
import plotly.graph_objs as go
from sklearn.cluster import KMeans 
from sklearn import datasets 
from sklearn.preprocessing import StandardScaler 

In [61]:
# copying the playlist just in case
hugeplaylist = huge_playlist.copy()

In [62]:
# dropping extra columns, leaving only the song to use later as index
hugeplaylist.drop(['artist', 'song_id', 'id', 'uri', 'track_href', 'mode', 'analysis_url', 
                   'duration_ms', 'time_signature', 'type'], axis = 1, inplace = True)

In [63]:
# Song as index! yay!
hugeplaylist.index = hugeplaylist.iloc[:,0]
# pick first column and turn it to index [:,_] - location

In [64]:
#keep rest of columns
hugeplaylist = hugeplaylist.iloc[:,1:]

In [65]:
# remove the header name
hugeplaylist.rename_axis(None,inplace=True)

### Scaling

In [66]:
hugeplaylist_scaled = StandardScaler().fit_transform(hugeplaylist)

In [67]:
hugeplaylist_scaled_df = pd.DataFrame(hugeplaylist_scaled,columns = ['danceability', 'energy', 
                                                                     'key', 'loudness', 
                                                                     'speechiness', 'acousticness', 
                                                                     'instrumentalness', 'liveness', 
                                                                     'valence', 'tempo'])

#### Using the 'right' number of clusters now

In [68]:
kmeans = KMeans(n_clusters = 52)

In [69]:
cluster = kmeans.fit(hugeplaylist_scaled_df)

In [70]:
#bring cluster into data frame
hugeplaylist_c = hugeplaylist.copy()
hugeplaylist_c['cluster'] = cluster.labels_

In [71]:
### Adding the cluster column to the original dataframe and reoganizing the data

In [72]:
huge_playlist['cluster'] = hugeplaylist_c['cluster'].values

In [179]:
# creating the cluster thingy
from IPython.core.display import display
from IPython.display import IFrame


def song_cluster(song):
    song = sp.search(q = song, limit = 1)
    song_id = song['tracks']
    song_id = song_id['items'][0]['id']
    song_features = sp.audio_features(song_id)
    song_data = pd.DataFrame(song_features, index= [0])
   # droping the numeric columns i didn't use
    song_data.drop(['mode', 'duration_ms', 'time_signature'], axis = 1, inplace = True)  
    song_num = song_data._get_numeric_data()
    song_scaled = StandardScaler().fit_transform(song_num)
    cluster = kmeans.predict(song_scaled)
    song_cluster = huge_playlist.loc[huge_playlist['cluster'] == int(cluster)]
    song_cluster.reset_index(drop=True, inplace=True) 
    song_name_id = random.choice(song_cluster["id"])
    songname = sp.track(song_name_id)['name']
    artist = sp.track(song_name_id)['artists'][0]['name']
    def recommend(song_name_id):
        display(IFrame(src=f"https://open.spotify.com/embed/track/{song_name_id}",
                        width="320",
                        height="80",
                        frameborder="0",
                        allowtransparency="true",
                        allow="encrypted-media",))
    return print("\033[1m" + 'That song is not hot right now. But listen to ' + songname + ' by ' + artist + ". I think you'll like it!."), recommend(song_name_id)

## The Ultimate Suggester

In [ ]:
# new suggester attempt

def second_suggester():
    song = str(input( "\033[1m" + """
    Welcome to the awesome song recommender!
    What song do you have in mind?: 
    """).lower().replace(" ", ''))
    # check hot 100
    check = billboard_top100[billboard_top100['song'].str.lower().str.replace(" ","").str.contains(song)]
    # get index
    index = check.index.tolist()
    if len(check) == 0:
        song_cluster(song)
        song2 = str(input("\033[1m" + "Want to try again? [y/n]: ")).lower().replace(" ", '')
        if song2 == 'y':
            print("\033[1m" + 'Awesome! Here, have another go:')
            second_suggester()
        elif song2 != 'n':
            print("\033[1m" + "Oh, come on, that's not an answer")
            
        elif song2 == 'n':
            print("\033[1m" + 'Oh well, your bad, this is an awesome recommender')
            return
    else:
        answer1 = input("\033[1m" + "Are you thinking about '" + billboard_top100.song[index].values[0] + "' by '" + billboard_top100.artist[index].values[0] + "'? [y/n]: ")
        #make a song suggestion
        if answer1.lower() == 'y':
            suggestion = billboard_top100.sample().index.tolist()
            print("\033[1m" + "That's a hot song! Listen to '" + billboard_top100['song'][suggestion].item() + "' by '" + billboard_top100['artist'][suggestion].item() + "' I think you'll like it.")
            song2 = str(input("\033[1m" + "Want to try again? [y/n]: ")).lower().replace(" ", '')
            if song2 == 'y':
                print("\033[1m" + 'Awesome! Here, have another go:')
                second_suggester()
            elif song2 != 'n':
                print("\033[1m" + "Oh, come on, that's not an answer")
                song2
            elif song2 == 'n':
                print("\033[1m" + 'Oh well, your bad, this is an awesome recommender')
                return
        elif answer1.lower() != 'n':
            print("\033[1m" + "Oh, come on, that's not an answer")
            song2 = str(input("\033[1m" + "Want to try again? [y/n]: ")).lower().replace(" ", '')
            second_suggester()
        elif answer1.lower() == 'n':
            song_cluster(song)
            song2 = str(input("\033[1m" + "Want to try again? [y/n]: ")).lower().replace(" ", '')
            if song2 == 'y':
                print("\033[1m" + 'Awesome! Here, have another go:')
                second_suggester()
            elif song2 != 'n':
                print("\033[1m" + "Oh, come on, that's not an answer")
                song2
            elif song2 == 'n':
                print("\033[1m" + 'Oh well, your bad, this is an awesome recommender')
                return
        elif song2 != 'n':
            print("\033[1m" + "Oh, come on, that's not an answer")
            song2
        elif song2 == 'n':
            print("\033[1m" + 'Oh well, your bad, this is an awesome recommender')
            return
second_suggester()